# exp15_minprj

>author : bae hueng myoung<br>
>date : 23 nov 2021


# Abstract
    영화 아이언맨에 등장하는 자비스는 토니스타크의 인공지능 비서로써 마치 사람처럼 쌍방향 커뮤니케이션의 능력을 보여준다. 토니스타크가 지시하면 자비스는 해당 말의 의미를 이해하고 즉각 실행에 옮기며 더 나은 대안을 추천하기도 한다. 이와 같이 기계와 사람이 터미널을 열어 소통하지 않고 사람처럼 하이레벨 랭귀지로 소통할 수 있다면 얼마나 편리할까? 
    
    기존에 시행했었던 문장 완성하기는 단어들의 유사성을 토대로 그럴듯한 문장을 출력하는 것에 불과하였다. 이전 노드에서 진행했었던 문장완성하기는 문법의 완성도에서 처참한 결과들을 보여주었다. 하지만 이번 트랜스포머는 자연스러운 문장의 완성과 함께 입력받은 문장에 대해서 적절한 문장을 출력하여 자연스러운 대화를 이끌어내도록 모델을 설계할 것이다.

# work flow

주요 트랜스포머 설계 과정들<br>
Positional Encoding Layer design<br>
scaled dot product attention<br>
multi-head attention<br>
masking<br>
enc design<br>
dec design<br>




In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

## Positional Ecoding Layer

In [2]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

## scaled dot product

In [3]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output


## multi-head attention

In [4]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs

## Masking

모든 문장의 길이는 최대 문장 길이와 항상 동일하지 않다. 이러한 문장 길이의 상이함은 모델 학습에 문제가 된다. 

In [6]:
# zero padding loacation 

def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [7]:
# look ahead mask
# ouput "1" in look ahead mask processed part

def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)


print(create_look_ahead_mask(tf.constant([[1, 2, 3, 4, 5]])))

tf.Tensor(
[[[[0. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 0. 1. 1.]
   [0. 0. 0. 0. 1.]
   [0. 0. 0. 0. 0.]]]], shape=(1, 1, 5, 5), dtype=float32)


In [8]:
print(create_look_ahead_mask(tf.constant([[0, 5, 1, 5, 5]])))

tf.Tensor(
[[[[1. 1. 1. 1. 1.]
   [1. 0. 1. 1. 1.]
   [1. 0. 0. 1. 1.]
   [1. 0. 0. 0. 1.]
   [1. 0. 0. 0. 0.]]]], shape=(1, 1, 5, 5), dtype=float32)


## enc design

In [9]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [10]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

슝=3


## dec design

In [11]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [12]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

## getting of chatbotdata.cvs

한글 챗봇 데이터는 총 11823개의 query와 answer의 pair로 구성되어 있다. 이 데이터들을 토대로 특정 쿼리에 대해서 적절한 answer를 출력할 수 있는 트랜스포머를 만들어볼 것이다.

In [36]:
import pandas as pd
file_path = os.getenv('HOME') + '/aiffel/exp15_transformer/data/ChatbotData.csv'
df = pd.read_csv(file_path)
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [40]:
df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [41]:
# 사용할 샘플의 최대 개수
MAX_SAMPLES = len(df)
print(MAX_SAMPLES)

11823


In [58]:
# 전처리 함수
def preprocess_sentence(sentence):

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  # 예를 들어서 "I am a student." => "I am a student ."와 같이
  # student와 온점 사이에 거리를 만듭니다.
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r"[^a-zA-Zㄱ-ㅣ가-힣0-9?.!,]+", " ", sentence)
  sentence = sentence.strip()
  return sentence

In [47]:
questions = list(df['Q'])
answers = list(df['A'])
len(answers)

11823

In [52]:
questions[:5]

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']

In [59]:
prepro_questions = []
prepro_answers = []

for i in range(len(questions) - 1):
  # 전처리 함수를 질문에 해당되는 inputs와 답변에 해당되는 outputs에 적용.
  prepro_questions.append(preprocess_sentence(questions[i]))
  prepro_answers.append(preprocess_sentence(answers[i]))

prepro_questions[:5]

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'ppl 심하네']

#### 전처리는 기존 lms의 경우 영문과 다른 조건을 제외한 나머지 기호들을 모두 제거하는 기능을 수행하였다. 하지만 이번 cvs file은 한글 챗봇이므로 regular expression에서 알파벳에  'ㄱ-ㅣ가-힣0-9'를 추가하여 한글과 나머지 조건을 제외한 모든 기호들을 제거하였다. 이러한 전처리 후의 결과는 위와 같다.

In [60]:
print('전체 샘플 수 :', len(prepro_questions))
print('전체 샘플 수 :', len(prepro_answers))

전체 샘플 수 : 11822
전체 샘플 수 : 11822


In [61]:
print('전처리 후의 22번째 질문 샘플: {}'.format(prepro_questions[21]))
print('전처리 후의 22번째 답변 샘플: {}'.format(prepro_answers[21]))

전처리 후의 22번째 질문 샘플: 가스비 장난 아님
전처리 후의 22번째 답변 샘플: 다음 달에는 더 절약해봐요 .


## Toeknizer design

In [62]:
import tensorflow_datasets as tfds

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성. (Tensorflow 2.3.0 이상) (클라우드는 2.4 입니다)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [64]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [65]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8170]
END_TOKEN의 번호 : [8171]


In [66]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8172


In [67]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(prepro_questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(prepro_answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [5759, 607, 2490, 4160]
정수 인코딩 후의 21번째 답변 샘플: [2353, 7510, 5, 6273, 94, 1468]


In [89]:
max_questions_len = 0
for sentence in prepro_questions:
    length = len(sentence.split(' '))
    if max_questions_len < length:
        max_questions_len = length
        
max_questions_len

16

In [90]:
max_answers_len = 0
for sentence in prepro_answers:
    length = len(sentence.split(' '))
    if max_answers_len < length:
        max_answers_len = length
        
max_answers_len

24

In [91]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
# 위의 샘플 최대길이는 24로 집계되었으므로 최대 길이는 +2인 26으로 한다
MAX_LENGTH = 26
print(MAX_LENGTH)

26


In [96]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 26 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 26으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

In [93]:
filtered_questions, filtered_answers = tokenize_and_filter(prepro_questions, prepro_answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(filtered_questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(filtered_answers)))

단어장의 크기 : 8172
필터링 후의 질문 샘플 개수: 11817
필터링 후의 답변 샘플 개수: 11817


In [97]:
# teacher forcing
# sample length가 11817이므로 shuffle이 제대로 이루게 하기 위해서 buffer size는 이보다 같거나 크게 설정하여야 한다
BATCH_SIZE = 64
BUFFER_SIZE = 120000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': filtered_questions,
        'dec_inputs': filtered_answers[:, :-1]
    },
    {
        'outputs': filtered_answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

## model design & trainig

In [98]:
# transformer function definition

def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [99]:
# model design

tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3146240     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [100]:
# loss function

def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [101]:
# learning rate

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [102]:
## model compile

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [103]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
185/185 [==============================] - 25s 92ms/step - loss: 2.3577 - accuracy: 0.0224
Epoch 2/20
185/185 [==============================] - 17s 92ms/step - loss: 1.9432 - accuracy: 0.0769
Epoch 3/20
185/185 [==============================] - 17s 92ms/step - loss: 1.5985 - accuracy: 0.0784
Epoch 4/20
185/185 [==============================] - 17s 92ms/step - loss: 1.4495 - accuracy: 0.0840
Epoch 5/20
185/185 [==============================] - 17s 92ms/step - loss: 1.3614 - accuracy: 0.0888
Epoch 6/20
185/185 [==============================] - 17s 92ms/step - loss: 1.2722 - accuracy: 0.0957
Epoch 7/20
185/185 [==============================] - 17s 92ms/step - loss: 1.1767 - accuracy: 0.1051
Epoch 8/20
185/185 [==============================] - 17s 92ms/step - loss: 1.0497 - accuracy: 0.1178
Epoch 9/20
185/185 [==============================] - 17s 92ms/step - loss: 0.9291 - accuracy: 0.1317
Epoch 10/20
185/185 [==============================] - 17s 92ms/step - loss: 0.802

## Inference

In [104]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)

In [105]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

In [108]:
def sentence_generating(input_sentences):
    for sentence in input_sentences:
        sentence_generation(sentence)

In [112]:
talker = ['밥 먹었어요?','너무 바쁜 날이야','언제 취업할 수 있을까?','세상의 본질은 무엇일까?','어떤 영화가 좋을까?']
sentence_generating(talker)

입력 : 밥 먹었어요?
출력 : 저랑 놀아요 .
입력 : 너무 바쁜 날이야
출력 : 일상으로 돌아갑시다 .
입력 : 언제 취업할 수 있을까?
출력 : 글쎄요 .
입력 : 세상의 본질은 무엇일까?
출력 : 직접 물어보는 것도 나쁘지 않아요 .
입력 : 어떤 영화가 좋을까?
출력 : 그 사람도 당신을 좋아하겠어요 .


# analysis : 

    트랜스포머를 이용하여 적절한 질문에 적절한 답변을 출력해보는 모델을 만들었고 결과는 위와 같다. 첫번째 질문은 '밥 먹었어요?'라는 질문에 대하여 '저랑 놀아요'라는 다소 주의가 결핍된 어린이가 내뱉는듯한 답변을 내놓았다. 두번째는 '너무 바쁜 날이야'라는 말에 대하여 '일상으로 돌아갑시다'의 답변으로 상대방의 심리나 상황에 대한 이해도가 낮은 기계적인 답변의 결과를 보여주었다. 이러한 결과는 모델이 문장을 '문법'적으로 완성도에 대해서 큰 문제가 없는 성능을 보여주었지만 상대방의 대화에서 의미를 추출하고 이해하는 고등적인 처리에서는 많은 한계점을 보이고 있다. 
    
    이러한 한계점은 축적된 이해도가 부족하기 때문이다. 트랜스포머를 이용한 encoder, decoder 설계를 살펴보면 문장에 대한 이해도에 대한 설계는 전무하다. 트랜스포머는 문장 단위들로 한꺼번에 입력받아 문장 내 단어간 유사도, 문장과 문장의 유사도 등의 기계적인 수치에 의존한다. 이러한 계산은 각 단어가 갖는 함축된 의미를 이해하는데 설계되었다고 볼 수 없다. 사람은 단어 "바다"를 입력받으면 곧장 머릿속으로 자신이 경험했던 바다의 각종 이미지, 소리, 느낌 등의 다양한 정보들을 머릿속에 로딩한다. 더군다나 퀴즈게임과 같이 보상과 관련되어 있다면 주의력(attention)을 끌어올려 더욱 더 상세한 정보들을 떠올린다. 하지만 기계는 단어가 갖는 정보에 대해서 방대한 정보들을 로딩하지 않으며 유사도의 수치만 갖고 있다. 따라서 이번 프로젝트의 챗봇은 기계가 사람의 문장을 이해하여 출력했다고 보기 어렵다. 
    
    하지만 이전 노드와 다르게 문장의 완성도는 매우 준수한 결과를 낳았다. 출력된 문장 단독으로 보았을 때 문법에 전혀 흠이 없었으며 자연스럽다. 이러한 이유는 트랜스포머가 적용된 것이 크다. 트랜스포머는 문장 내, 문장간 유사도에 대해 보다 더 넓게 계산되어진다. 문장 내 단어들간의 유사성에 대해 더 많이 참고하고 적절한 문장을 완성하는데 효과적인것으로 보인다. 더군다나 멀티헤드 어텐션기법으로 문장 내 단어들의 유사도에 대해서 놓칠 수 있는 부분을 보완하는 정밀성까지 더해졌다. 


# Limits :

learning rate를 조절하여 모델의 성능을 시험<br>
각 hyperparameter들을 변경해보고 어떤 변화가 있었으며 각 파라미터들은 모델성능에 어떤 연관성이 있는지 시험<br>
문장의 입력과 출력에서 유사도의 계산 과정을 파악해보는 실험<br>
멀티헤드어텐션을 적용하였을 때와 그렇지 않았을 때의 유사도 차이를 실험





# In my opinion

    트랜스포머 기법은 매우 복잡해보이지만 RNN의 기초를 공부할 때 전체적인 설계 의도성을 파악하는데 용이한것 같다. lms에 올려진 교육 자료들은 트랜스포머에 대한 심도깊은 이해보다 코딩방법과 메뉴얼에 치중한듯한 느낌을 받았다. 하지만 리드퍼실(이영빈님)의 자료는 더 방대한 레퍼런스들이 있었고 이러한 내용들을 이해하기 위해선 RNN부터 자세히 살펴보고 설계목적과 기존 RNN, LSTM의 한계성(gradient문제, 유사도문제)등에 대해서 자세히 알 필요가 있다고 느꼈다. 아직 트랜스포머에 대해서 완벽하게 이해한 것은 아니나 사람과 대화 가능한 자연어 처리에 공을 들인 엔지니어들의 아이디어와 설계방향은 나에게 많은 감동을 주었다.(물론 나는 NLP가 아닌 CV를 선택했다.)